In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import psycopg2
import streamlit as st

In [2]:
#Connect to local postgres DB
conn = psycopg2.connect(**st.secrets["postgres"])
cur = conn.cursor()

In [3]:
def run_query(query_path):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

## Questions To Answer:

- What is the proportion between equity and fixed income over time?
- What is the proportion between sectors over time?
- What is the proportion between regions over time?
- What are the equity inflows over time? Assessing ownership %
- What sectors has the fund been a buyer in?
- What regions/countries has the fund been buying

### Equity and Fixed Income Proportions Over Time

In [4]:
equity_fi_df = run_query('SQL/static/eq_fi_proportions.sql')
equity_fi_df

,year,Equity Proportion,Fixed Income Proportion
0,1998,39.66,60.34
1,1999,41.87,58.13
2,2000,38.13,61.87
3,2001,40.78,59.22
4,2002,36.74,63.26
5,2003,42.91,57.09
6,2004,39.61,60.39
7,2005,46.47,53.53
8,2006,38.18,61.82
9,2007,45.81,54.19


### Sector Proportions Over Time

In [5]:
sector_prop_df = run_query('SQL/static/sector/sector_proportions.sql')
sector_prop_df

,year,category,Sector,Proportion of Fund
0,1998,Equity,Basic Materials,1.67
1,1998,Equity,Consumer Discretionary,4.69
2,1998,Equity,Consumer Staples,4.68
3,1998,Equity,Energy,2.23
4,1998,Equity,Financials,9.49
...,...,...,...,...
370,2022,Equity,Utilities,1.87
371,2022,Fixed Income,Corporate Bonds,7.24
372,2022,Fixed Income,Government Bonds,3.07
373,2022,Fixed Income,Securitized Bonds,1.27


### Region Proportions Over Time

In [6]:
region_prop_df = run_query('SQL/static/region/region_proportions.sql')
region_prop_df

,year,Region,proportion
0,1998,Asia,18.77
1,1998,Europe,50.76
2,1998,International,2.63
3,1998,Latin America,0.10
4,1998,Middle East,0.03
...,...,...,...
186,2022,International,0.73
187,2022,Latin America,0.99
188,2022,Middle East,0.44
189,2022,North America,46.06


### Ownership By Sector Over Time

In [7]:
sector_ownership_df = run_query('SQL/static/sector/sector_ownership.sql')
sector_ownership_df

,year,Sector,avg_percent_ownership
0,1998,Basic Materials,0.05
1,1998,Consumer Discretionary,0.05
2,1998,Consumer Staples,0.05
3,1998,Energy,0.04
4,1998,Financials,0.05
...,...,...,...
269,2022,Industrials,1.11
270,2022,Real Estate,1.33
271,2022,Technology,1.05
272,2022,Telecommunications,1.01


### Cumulative Change In Percent Ownership By Sector - Last 10 Years

In [8]:
ownership_change_sector_ten_df = run_query('SQL/static/sector/ownership_change_sector_ten_years.sql')
ownership_change_sector_ten_df

,Sector,cumulative_bp_change_of_ownership
0,Real Estate,0.30
1,Telecommunications,0.17
2,Basic Materials,0.15
3,Industrials,0.15
4,Health Care,0.14
5,Technology,0.14
6,Financials,0.12
7,Consumer Discretionary,0.09
8,Utilities,0.04
9,Consumer Staples,0.03


### Cumulative Change In Percent Ownership and Market Value - Last 10 Years

In [9]:
mrkt_value_ownership_change_sector_ten_years_df = run_query('SQL/static/sector/mrkt_value_ownership_change_sector_ten_years.sql')
mrkt_value_ownership_change_sector_ten_years_df

,Sector,Cumulative Average Ownership Change,Cumulative Market Value Percent Change
0,Real Estate,0.30,136.14
1,Telecommunications,0.17,39.36
2,Basic Materials,0.15,7.13
3,Industrials,0.15,59.47
4,Health Care,0.14,103.41
5,Technology,0.14,131.77
6,Financials,0.12,37.91
7,Consumer Discretionary,0.09,91.25
8,Utilities,0.04,34.78
9,Consumer Staples,0.03,-32.95


### Cumulative Change In Percent Ownership By Region - Last 10 Years

In [10]:
ownership_change_region_ten_df = run_query('SQL/static/region/ownership_change_region_ten_years.sql')
ownership_change_region_ten_df

,Region,cumulative_bp_change_of_ownership
0,Oceania,0.55
1,Africa,0.48
2,Middle East,0.46
3,Latin America,0.25
4,Asia,0.20
5,North America,0.13
6,Europe,-0.10


### Dynamic Queries - Testing Zone

In [11]:
#Dynamic query function for sector

def run_query_dynamic(query_path, num_years):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query, (num_years,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

### Sector Dynamic

In [12]:
#Enter the number of years to go back

ownership_change_sector_dynamic_df = run_query_dynamic('SQL/dynamic/ownership_change_sector.sql',3)
ownership_change_sector_dynamic_df

,Sector,cumulative_bp_change_of_ownership
0,Real Estate,-0.06
1,Utilities,-0.07
2,Telecommunications,-0.08
3,Health Care,-0.12
4,Financials,-0.12
5,Technology,-0.13
6,Basic Materials,-0.17
7,Industrials,-0.17
8,Energy,-0.17
9,Consumer Staples,-0.20


### Region Dynamic

In [13]:
ownership_change_region_dynamic_df = run_query_dynamic('SQL/dynamic/ownership_change_region.sql',7)
ownership_change_region_dynamic_df

,Region,cumulative_bp_change_of_ownership
0,Oceania,0.29
1,Europe,0.21
2,Middle East,0.18
3,Latin America,0.16
4,Africa,0.04
5,North America,0.03
6,Asia,-0.11


### Top 10 Companies Dynamic

In [14]:
ownership_change_company_dynamic_df = run_query_dynamic('SQL/dynamic/top10_ownership_change_company.sql',3)
ownership_change_company_dynamic_df

,Company,Sector,cumulative_bp_change_of_ownership
0,Rogers Corp,Technology,7.82
1,Neogen Corp,Health Care,6.74
2,Hudson Pacific Properties Inc,Real Estate,6.35
3,Rayonier Inc,Real Estate,6.35
4,boohoo Group PLC,Consumer Discretionary,6.31
5,UDR Inc,Real Estate,6.27
6,Vonovia SE,Real Estate,5.88
7,Invitation Homes Inc,Real Estate,5.43
8,American Homes 4 Rent,Real Estate,5.01
9,Grainger PLC,Real Estate,4.82


### Top 10 Countries Dynamic

In [15]:
ownership_change_country_dynamic_df = run_query_dynamic('SQL/dynamic/top10_ownership_change_country.sql',3)
ownership_change_country_dynamic_df

,Country,cumulative_bp_change_of_ownership
0,Qatar,0.98
1,Bangladesh,0.55
2,Kuwait,0.44
3,Finland,0.29
4,Hungary,0.29
5,Lithuania,0.22
6,Turkey,0.20
7,Moldova,0.17
8,Tunisia,0.17
9,Brazil,0.13


### Country Dynamic - Multiselect

In [16]:
def run_query_dynamic_country(query_path, num_years, countries):
    with open(query_path, 'r') as file:
        query = file.read()
    placeholders = ','.join(['%s'] * len(countries))
    formatted_query = query.format(placeholders)
    with conn.cursor() as cur:
        cur.execute(formatted_query, (num_years, *countries))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)


### Average Ownership

In [17]:
run_query_dynamic_country('SQL/dynamic/avg_ownership_country_multiselect.sql',3,['United States','Canada'])

,year,Country,avg_percent_ownership
0,2019,Canada,1.03
1,2019,United States,1.06
2,2020,Canada,0.98
3,2020,United States,0.99
4,2021,Canada,0.77
5,2021,United States,0.82
6,2022,Canada,0.83
7,2022,United States,0.87


### Cumulative Basis Point Change

In [18]:
ownership_change_country_dynamic_df = run_query_dynamic_country('SQL/dynamic/ownership_change_country_multiselect.sql', 2, ['United States', 'Canada', 'Mexico', 'United Kingdom',
                                                                                                                             'Qatar','France','Germany'])
ownership_change_country_dynamic_df


,Country,cumulative_bp_change_of_ownership
0,Qatar,0.34
1,Germany,0.12
2,Mexico,0.00
3,France,-0.07
4,United Kingdom,-0.08
5,United States,-0.12
6,Canada,-0.15


### Top 10 Companies By Basis Point Change

### 

In [19]:
top10_ownership_change_company_dynamic_df = run_query_dynamic_country('SQL/dynamic/top10_ownership_change_company_multiselect.sql', 3, ['Canada'])
top10_ownership_change_company_dynamic_df

,Company,cumulative_bp_change_of_ownership
0,Northland Power Inc,2.26
1,Algonquin Power & Utilities Corp,1.79
2,BlackBerry Ltd,1.12
3,Cargojet Inc,0.74
4,Aritzia Inc,0.69
5,Canadian Pacific Railway Ltd,0.62
6,Wheaton Precious Metals Corp,0.51
7,GFL Environmental Inc,0.49
8,Home Capital Group Inc,0.41
9,Teck Resources Ltd,0.41
